# Digital Twin Fairways
WIP to use the OpenTNSim engine

In [1]:
import datetime
import json
import logging
import time

import dtv_backend.berthing

# the simpy processes and objects
import dtv_backend.compat

# library to load the fairway information network
import dtv_backend.fis
import dtv_backend.network
import dtv_backend.network.network_utilities
import dtv_backend.postprocessing
import dtv_backend.simple
import dtv_backend.simulate
import geojson
import networkx as nx
import pandas as pd
import shapely
import simpy
from networkx.readwrite import json_graph

# opentnsim import
from opentnsim import core

# reload for debugging purposes
%load_ext autoreload
%autoreload 2

logger = logging.getLogger("notebook")

/Users/baart_f/.virtualenvs/main/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning:

The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.



### Input
You can define your input in a json configuration file. The relevant parts are sites, fleet and climate.

In [2]:
# example input
with open("../../dtv_backend/tests/user/2022-09-05.geojson") as f:
    config = geojson.load(f)

In [3]:
print("\n".join(config['fleet'][0]['properties'].keys()))

CEMT-class
RWS-class
Vessel type
Description (English)
Description (Dutch)
Beam [m]
Beam 10% percentile [m]
Beam 50% percentile [m]
Beam 90% percentile [m]
Length [m]
Length 10% percentile [m]
Length 50% percentile [m]
Length 90% percentile [m]
Draught loaded [m]
Draught average [m]
Draught empty [m]
Height average [m]
Load Weight average [ton]
Load weight maximum [ton]
speed
image
capacity
name
count
Velocity [m/s]
Engine power maximum [kW]


### Simulate with ``v3_run``

In [4]:
def v3_run(config):
    """run a simulation using the opentnsim compatibility kernel"""
    logger.info("Running simulation 👩‍💻")
    # Run for n days

    env = dtv_backend.simulate.create_env(config)
    ports = dtv_backend.simulate.create_ports(env, config)
    ships = dtv_backend.simulate.create_ships(env, config)
    operator = dtv_backend.simulate.create_operator(env, ships, ports, config)

    n_days = 60
    n_days_in_future = env.epoch + datetime.timedelta(days=n_days)
    env.run(until=n_days_in_future.timestamp())
    result = {
        "env": env,
        "operator": operator,
        "ships": ships,
        "config": config,
        "ports": ports,
    }

    return result


In [5]:
def v3_dtv_backend_simulate(config):
    result = v3_run(config)
    env = result['env']
    logbook = result['operator'].logbook
    log_df = pd.DataFrame(logbook)
    log_json = dtv_backend.postprocessing.log2json(log_df)
    
    response = {
        "log": log_json,
        "config": config,
        "env": {
            "epoch": env.epoch.timestamp(),
            "epoch_iso": env.epoch.isoformat(),
            "now": env.epoch,
            "now_iso": datetime.datetime.fromtimestamp(env.now).isoformat(),
        },
    }
    return response
    

In [6]:
resp = v3_dtv_backend_simulate(config)

/Users/baart_f/src/digitaltwin-waterway/dtv_backend/dtv_backend/fis.py:118: ShapelyDeprecationWarning:

The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.



AssertionError: 